In [57]:
import os
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression




In [59]:

# Load the training and test data
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'

row_ids = df_test['row ID']
df_test.drop(['row ID'], axis=1, inplace=True)

numeric_col = df_test.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df_test.select_dtypes(exclude=["number"]).columns.tolist()

# Label encode the "sub_area" column
label_encoder = LabelEncoder()
df_train['sub_area'] = label_encoder.fit_transform(df_train['sub_area'])
df_test['sub_area'] = label_encoder.transform(df_test['sub_area'])

# Create dummy variables for categorical features
X_train = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

X_test = pd.get_dummies(df_test, drop_first=True)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
scaler = StandardScaler()
X_train[numeric_col] = scaler.fit_transform(X_train[numeric_col])
X_test[numeric_col] = scaler.transform(X_test[numeric_col])

print('X_train_scaled shape:', X_train.shape)
print('X_test_scaled shape:', X_test.shape)

all_columns = X_train.columns.tolist()

# Get the non-numeric columns by subtracting numeric_col from all_columns
non_numeric_columns = [col for col in all_columns if col not in numeric_col]


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train.dtypes


In [ ]:

# Step 2: Use a Decision Tree Regressor to get the 100 most important features
tree_regressor = DecisionTreeRegressor(max_depth=8, random_state=42)
tree_regressor.fit(X_train, y)
importances = tree_regressor.feature_importances_


# Get indices of the top 100 features
top_100_feature_indices = np.argsort(importances)[-150:]

# Select the top 100 features
X_train = X_train.iloc[:, top_100_feature_indices]
X_test = X_test.iloc[:, top_100_feature_indices]


In [ ]:
selector = SelectKBest(score_func=f_regression, k=100)

X_train = selector.fit_transform(X_train,y)


X_test = selector.transform(X_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 
print('X_train_scaled shape:', X_train.shape)
print('X_test_scaled shape:', X_test.shape)

In [ ]:
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print('X_train_pca shape:', X_train.shape)
print('X_test_pca shape:', X_test.shape)


In [ ]:
threshold = 1

selector = VarianceThreshold(threshold=threshold)

X_train = selector.fit_transform(X_train)
X_test = selector.transform(X_test)


print(X_train.shape)
print(X_test.shape)

In [ ]:
# Step 3: Use Forward Selection to get the first 10 best features
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)
selector.fit(X_train, y)

# Get the selected feature indices
selected_feature_indices = selector.get_support()


In [ ]:

# # Select the first 10 best features
# X_train = X_train.iloc[:, selected_feature_indices]
# X_test = X_test.iloc[:, selected_feature_indices]
# Select the first 10 best features
X_train = X_train[:, selected_feature_indices]
X_test = X_test[:, selected_feature_indices]

print('X_train_top10 shape:', X_train.shape)
print('X_test_top10 shape:', X_test.shape)



In [ ]:

# Step 4: Use Polynomial Features with interaction on the selected features
poly = PolynomialFeatures(degree=3, interaction_only=True)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

print('X_train_poly shape:', X_train.shape)
print('X_test_poly shape:', X_test.shape)


In [ ]:
print(X_train.shape)

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Define your model with L2 regularization
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1)
])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model with RMSprop optimizer
optimizer = RMSprop(learning_rate=0.001)  # You can adjust the learning rate
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=[keras.metrics.RootMeanSquaredError(), 'mae'])

# Fit the model
model.fit(X_train, y, epochs=15, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])


In [ ]:

model = Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    
    keras.layers.Dense(1)
])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer=custom_optimizer,
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])

model.fit(X_train, y, epochs=15, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam, legacy
import math


# define the keras model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dropout(0.2))  # Apply dropout with a rate of 0.2
model.add(Dense(64, activation='relu'))#sigmoid
# model.add(Dense(64, activation='relu'))#sigmoid
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Add regularizers to the dense layers
model.layers[1].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
model.layers[2].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
#model.layers[3].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01

opt = legacy.Adam(learning_rate=0.00021)  

# compile the keras model
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['RootMeanSquaredError'])
#apply early stoppping
early_stopping_monitor = EarlyStopping(patience=5)
#apply model checkpoint

#fit the keras model on the dataset
model.fit(X_train, y, epochs=50, batch_size=63, validation_split=0.2, callbacks=[early_stopping_monitor])

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
test_predictions = model.predict(X_test).flatten()

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

result_df.to_csv('predictions_55.csv', index=False)




# Shahood




In [ ]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'

row_ids = df_test['row ID']
df_test.drop(['row ID'], axis=1, inplace=True)


X_train = df_train.drop(columns=[target_col])
y = df_train[target_col]

X_test = df_test


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (181507, 271)
X_test shape: (77789, 271)


In [ ]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

#'''
# Apply label encoding to each categorical column
for col in categorical_cols:
    le = LabelEncoder()
    # Fill missing values with a placeholder string
    #X_train[col] = X_train[col].fillna('Missing')
    X_train[col] = le.fit_transform(X_train[col])
    #X_test[col] = X_test[col].fillna('Missing')
    X_test[col] = le.transform(X_test[col])
    
    
#apply minmax scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape[1])

271


In [ ]:

# Perform PCA to reduce dimensionality to 240 components
n_components = 240
pca = PCA(n_components=n_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Now X_train_pca and X_test_pca contain the top 240 principal components
print(X_train.shape[1])

240


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=43)

In [ ]:
X_train.shape[1]

271

In [62]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from keras.optimizers import legacy

# define the keras model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dropout(0.2))  # Apply dropout with a rate of 0.2
model.add(Dense(64, activation='relu'))#sigmoid
#model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Add regularizers to the dense layers
model.layers[1].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
model.layers[2].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
#model.layers[3].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01

opt = legacy.Adam(learning_rate=0.002)  

# compile the keras model
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['RootMeanSquaredError'])
#apply early stoppping
early_stopping_monitor = EarlyStopping(patience=5)
#apply model checkpoint

#fit the keras model on the dataset
model.fit(X_train, y_train, epochs=50, batch_size=96, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

Epoch 1/50
1513/1513 [==============================] - 2s 1ms/step - loss: 425143914463232.0000 - root_mean_squared_error: 20619020.0000 - val_loss: 182914889285632.0000 - val_root_mean_squared_error: 13524603.0000
Epoch 2/50
1513/1513 [==============================] - 1s 907us/step - loss: 186063150645248.0000 - root_mean_squared_error: 13640497.0000 - val_loss: 177905363582976.0000 - val_root_mean_squared_error: 13338117.0000
Epoch 3/50
1513/1513 [==============================] - 1s 888us/step - loss: 182619811610624.0000 - root_mean_squared_error: 13513690.0000 - val_loss: 175783582629888.0000 - val_root_mean_squared_error: 13258340.0000
Epoch 4/50
1513/1513 [==============================] - 1s 913us/step - loss: 180844194955264.0000 - root_mean_squared_error: 13447832.0000 - val_loss: 174482626969600.0000 - val_root_mean_squared_error: 13209187.0000
Epoch 5/50
1513/1513 [==============================] - 1s 949us/step - loss: 179634809339904.0000 - root_mean_squared_error: 1340

In [63]:
#calculate rmse
y_val_pred = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print('Validation RMSE:', val_rmse)

1135/1135 [==============================] - 1s 432us/step
Validation RMSE: 13045160.650230566


In [64]:
test_predictions = model.predict(X_test).flatten()

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

result_df.to_csv('predictions_59.csv', index=False)




2431/2431 [==============================] - 1s 470us/step
